In [2]:
import backtrader as bt
import pandas as pd

btc_csv_parsed = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-03_UTC_with_lookback.csv",
    dtformat=('%Y-%m-%d %H:%M:%S'),
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1
)